# 综述
pLSA:[Probabilistic latent semantic analysis](https://arxiv.org/pdf/1301.6705.pdf)，EM算法，较为简单，容易过拟合（M步存在参数：$p(z_k|d_m)$，根训练样本完全相关）

LDA: 参考[lda-LDA数学八卦](http://bloglxm.oss-cn-beijing.aliyuncs.com/lda-LDA%E6%95%B0%E5%AD%A6%E5%85%AB%E5%8D%A6.pdf ):第45页很形象（与pLSA区别：topic属于一个Dirichlet分布），Gibbs Sample（推导用了一些近似方法）。短文本实际效果并不太好

plsa泛化能力不好，主题分布是确定的，频率派参数估计基于已有数据。

DA数学推理优秀，但是实际用的时候，gibbs sampling公式其实是一个近似正比，效果也没有非常理想。反而plsa简单容易并行化。


# 工具
gensim，百度也开源了LDA工具，LDA不像word2vec，需要去除停用词
https://radimrehurek.com/gensim/models/ldamodel.html

# Ref
[NLP —— 图模型（三）pLSA（Probabilistic latent semantic analysis，概率隐性语义分析）模型](https://www.cnblogs.com/Determined22/p/7237111.html)

实践可以参考：https://www.cnblogs.com/chenbjin/p/5635853.html



In [10]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import gensim.models.ldamulticore
from gensim.models import LdaModel,LdaMulticore

# Create a corpus from a list of texts

common_dictionary = Dictionary(common_texts)

common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# Train the model on the corpus.

lda = LdaModel(common_corpus, num_topics=10)

In [13]:
# 速度更快
from gensim.test.utils import common_corpus, common_dictionary
lda = LdaMulticore(common_corpus, id2word=common_dictionary, num_topics=10)

In [12]:
lda.print_topics(10)

[(0,
  '0.500*"trees" + 0.045*"system" + 0.045*"response" + 0.045*"graph" + 0.045*"minors" + 0.045*"interface" + 0.045*"user" + 0.045*"human" + 0.045*"time" + 0.045*"survey"'),
 (1,
  '0.083*"trees" + 0.083*"graph" + 0.083*"system" + 0.083*"user" + 0.083*"interface" + 0.083*"minors" + 0.083*"human" + 0.083*"time" + 0.083*"response" + 0.083*"computer"'),
 (2,
  '0.083*"trees" + 0.083*"system" + 0.083*"graph" + 0.083*"user" + 0.083*"minors" + 0.083*"interface" + 0.083*"survey" + 0.083*"response" + 0.083*"human" + 0.083*"computer"'),
 (3,
  '0.083*"trees" + 0.083*"graph" + 0.083*"system" + 0.083*"user" + 0.083*"interface" + 0.083*"time" + 0.083*"survey" + 0.083*"eps" + 0.083*"response" + 0.083*"human"'),
 (4,
  '0.262*"minors" + 0.262*"graph" + 0.262*"survey" + 0.024*"trees" + 0.024*"system" + 0.024*"user" + 0.024*"interface" + 0.024*"human" + 0.024*"computer" + 0.024*"response"'),
 (5,
  '0.153*"computer" + 0.153*"survey" + 0.153*"response" + 0.153*"system" + 0.153*"time" + 0.153*"user" 